In [ ]:
import pandas as pd

data=pd.read_csv('Test Links.csv')
data.head()

,Source,Link,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7
0,Psychology Today,https://www.psychologytoday.com/us/basics/chil...,child development,developmental milestones,NaN,NaN,NaN,NaN,NaN
1,Psychology Today,https://www.psychologytoday.com/us/basics/adhd...,ADHD,attention,defiance,discipline,social interaction,self-esteem,sleep
2,Psychology Today,https://www.psychologytoday.com/us/basics/adop...,adoption,mental health,behavior challenge,NaN,NaN,NaN,NaN
3,Psychology Today,https://www.psychologytoday.com/us/basics/auti...,autism,behavioral therapy,social skills,discipline,treatment,NaN,NaN
4,Psychology Today,https://www.psychologytoday.com/us/blog/how-ra...,homework,rules,defiance,NaN,NaN,NaN,NaN


In [ ]:
links=data['Link']

Extracting Text from URL

In [ ]:
import csv

def extract_text(text,check):
    lines = text.split("\n")
    title_subtitle = lines[0] + ", " + lines[1]
    headings = []
    paragraphs = []
    heading = ""
    i=0
    while i<len(lines[2:]):
      while lines[i].startswith("H") and lines[i+1].startswith("H"):
        i+=1
      if lines[i].startswith("H") and lines[i+1].startswith("P"):
        heading=lines[i][3:]
        headings.append(heading)
        print("Inside H,P")
        print("heading "+ heading)
        heading=""
        i+=1
        paragraph = ""
        while lines[i].startswith("P") and lines[i+1].startswith("P"):
          print('Inside PP')
          print("paragraph "+ paragraph)
          paragraph+=lines[i]
          i+=1
        if lines[i].startswith("P") and lines[i+1].startswith("H"):
          print('Inside PH')
          print("paragraph "+ paragraph)
          paragraph+=lines[i]
          i+=1
        paragraphs.append(paragraph)
      i+=1
    return [(title_subtitle, h, p) for h, p in zip(headings, paragraphs)]

        


    # for line in lines[2:]:
    #     if line.startswith("H"):
    #         if heading:
    #             headings.append(heading)
    #         heading = line[3:]
    #         if paragraph:
    #             paragraphs.append(paragraph)
    #         paragraph = ""
    #     else:
    #         paragraph += line + " "
    # headings.append(heading)
    # paragraphs.append(paragraph)
    # return [(title_subtitle, h, p) for h, p in zip(headings, paragraphs)]

with open('ExtractedText.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['DocumentID', 'PassageID', 'Title', 'Subheading', 'Paragraph','URL'])

def store_output_in_csv(text,i,url,check):
    data = extract_text(text,check)
    with open('ExtractedText.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        DocumentID = i
        PassageID = 1
        for item in data:
            Title = item[0].split(",")[0][7:]
            #print("HHHHHHHHH"+Title + "\n")
            Subheading = item[1]
            #print("SHSHSHGSHG"+Subheading + "\n")
            Paragraph = item[2]
            Paragraph = Paragraph.replace("P:","")
            Paragraph = Paragraph.strip()
            if (not Paragraph.isspace()) and len(Paragraph)!=0  and  (not Subheading.isspace()) and len(Subheading)!=0 :
            #print("GGGGGGGGGGGGGGGGGGGG"+Paragraph + "\n")
              writer.writerow([DocumentID, PassageID, Title, Subheading, Paragraph,url])
              PassageID += 1

# convertion_csverror_list=[]            
# for i in range(89):
#     try:
#         with open("%d"%(i), "r", encoding="utf8") as f:         
#             text = f.read()
#             f1=open("{}_url".format(i),"r")
#             url=f1.read()
#             store_output_in_csv(text,i,url)
#     except:
#         convertion_csverror_list.append(i)

Individual Extraction

CompleteTest


In [ ]:
! pip install trafilatura 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from bs4 import BeautifulSoup
import trafilatura

# def get_data(url):
#     r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
#     return r.text

# final_text_BS4=[]

# with open('Final_Clean_URL_List.txt') as f:
#     lines = f.readlines()
URL=['https://childrensmd.org/browse-by-topic/parenting/strategies-parents-and-kids-help-prevent-bullying/',
     'https://kidshealth.org/en/parents/adhd-therapy.html#catlearning',
     'https://www.parents.com/toddlers-preschoolers/discipline/tantrum/a-parents-guide-to-temper-tantrums/',
     'https://www.psychologytoday.com/us/basics/autism/parenting-child-autism',
     'https://www.mother.ly/toddler/toddler-milestones/23-tantrum-taming-tips/',
     'https://childdevelopmentinfo.com/parenting/how-to-build-your-childs-confidence-and-self-esteem/',
     'https://www.healthychildren.org/English/ages-stages/toddler/Pages/Soothing-Your-Childs-Separation-Anxiety.aspx',
     'https://www.webmd.com/children/child-constipation-directory']

temp_list=[]
url=URL[0]



for i in range(1):
# for i,url in enumerate(links):
    #html_data = get_data(url)
    
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.content, 'html.parser')
    t=trafilatura.fetch_url(url)
    text=trafilatura.extract(t)
    text = text.split('\n')
    text=[s.strip() for s in text]
    print(text)

    
    try:
        title = soup.find("title").get_text()
        subtitle = soup.find("h1").get_text()
        subtitle=subtitle.strip()
        #data = "Title: " + title + "\nSubtitle: " + subtitle
        data="Title: " + title + "\nSubtitle: " + subtitle +"\n"
        seen=False
        head_seen=False
        Paragraph_seen=False

        for element in soup.find_all(["p", "h2", "h3","li","h4","h1"]):
            temp=element.get_text()

            if element.name in ["h1","h2","h3","h4"]:
                    head=element.get_text()
                    #if element.name == "li":
                        #data += "\n-LIST- " + element.get_text()+ "####"
                    head=head.replace("\n","")
                    head=head.strip()
                    if head in text: print("Found HEading++++++++++++++"+ " "+head)
                    if not head.isspace() and len(head)!=0:
                      data+="H: "+head+"\n"
                      seen=True
                      paragraph_counter=1
            # if temp.count('\n')<2:
            #     if element.name in ["h1","h2","h3","h4"]:
            #         head=element.get_text()

            #         #if element.name == "li":
            #             #data += "\n-LIST- " + element.get_text()+ "####"
            #         data+="H: "+head+"\n"
            #         seen=True

            else:
              if seen:
                body=element.get_text()
                        #if element.name in ["nav"]:
                                #data += "\n-LIST- " + element.get_text()+ "####"
                                #data += "\n MAINCONTENT" + element.name.capitalize() + ": " + element.get_text()
                body=body.replace("\n","")
                body=body.strip()
                if not body.isspace() and len(body)!=0:
                  # print("body->"+ str(body.strip().isspace())+body)
                  data+="P: "+str(paragraph_counter)+". "+body+"\n"
                  paragraph_counter+=1
                else:
                  # print("InsideElse")
                  continue
        print(data)
                
        # i = extract_text(data)
        # for g in i:
        #   print(g)
        # k = extract_text(data[-1])
        # print(k[-1].isspace())
        # print(len(k[-1])==0)
        # store_output_in_csv(data,i,url,text)

        #store_output_in_csv(data,i,url)
        # with open("{}".format(i), "w",encoding="utf-8") as fp:
        #     fp.write(''.join(data))
        # with open("{}_url".format(i), "w",encoding="utf-8") as fp1:
        #     fp1.write(''.join(url))
        #final_text_BS4.append(data)
    
    except:
        print("Error in URL: ",url)

z = extract_text(data,text)
for g in z:
  print(g)

print("END")

['Bullying in schools and social groups – including depersonalized social media platforms – seems to be rampant and still escalating. And in the strange, circular thought process of motherhood, I’ve inevitably brought it back to my kids: how can I be sure they aren’t the victims of bullying? Would they step in for someone who was being bullied? Have I taught them how important it is not to be a bully? What would it be like if they could live in a society without bullies or mean girls or trolls? What if the hurting and anxious and depressed among us were more often met with kindness than cruelty? How might that impact the world around us?', 'I’m not a psychologist. As an intensivist, I tend to see the tragic end result of things rather than their more subtle or insidious beginnings. In fact, at the beginning of the COVID-19 pandemic, one of my hopes for a silver lining was that when people saw the intensive care unit up close and unwelcome, they would begin to take more seriously the se